# 56. ステレオ視と視差
## Stereo Vision and Disparity

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] ステレオビジョンの原理と深度推定の仕組みを理解する
- [ ] ステレオ画像の平行化（Rectification）の目的と方法を説明できる
- [ ] 視差（Disparity）と深度の関係を数学的に記述できる
- [ ] ブロックマッチングによる視差計算を実装できる
- [ ] SGM（Semi-Global Matching）の概念を理解する
- [ ] 視差マップから深度マップを生成できる

---

## 前提知識

- 51: ピンホールカメラモデルと射影変換
- 55: エピポーラ幾何の理論
- 線形代数：ホモグラフィ、行列演算

**難易度**: ★★★★☆（上級）  
**推定学習時間**: 90-120分

---

## 1. ステレオビジョンとは

**ステレオビジョン**は、2台のカメラ（または1台のカメラの2つの位置）から撮影した画像を使って、シーンの3D構造（深度情報）を復元する技術です。

### 人間の両眼立体視との類似

人間の目は約6.5cm離れており、この差（両眼視差）から脳が深度を知覚します。ステレオビジョンは同じ原理をコンピュータで実現します。

### ステレオビジョンのワークフロー

```
1. カメラキャリブレーション
       ↓
2. ステレオ平行化（Rectification）
       ↓
3. 対応点探索（視差計算）
       ↓
4. 深度マップ生成
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, Optional
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

---

## 2. ステレオ幾何学の基礎

### 2.1 標準ステレオ配置

最も単純なステレオ配置は、2台のカメラが**平行**に配置され、**同一平面上**にある場合です。

- **ベースライン** $b$: 2つのカメラ中心間の距離
- **焦点距離** $f$: カメラの焦点距離（ピクセル単位）
- **画像面**: 平行かつ同一平面上

### 2.2 視差（Disparity）の定義

3D点 $\mathbf{X} = (X, Y, Z)$ が左画像で $(x_L, y_L)$、右画像で $(x_R, y_R)$ に投影されるとき：

$$d = x_L - x_R$$

これが**視差（Disparity）**です。

### 2.3 深度と視差の関係

標準ステレオ配置では：

$$Z = \frac{f \cdot b}{d}$$

ここで：
- $Z$: 深度（カメラからの距離）
- $f$: 焦点距離（ピクセル）
- $b$: ベースライン（実世界の単位、例：メートル）
- $d$: 視差（ピクセル）

**重要な洞察**: 視差は深度に**反比例**します。近いものほど視差が大きく、遠いものほど視差が小さい。

In [ ]:
def visualize_stereo_geometry():
    """ステレオ幾何学の可視化"""
    fig = plt.figure(figsize=(14, 6))
    
    # 3Dビュー
    ax1 = fig.add_subplot(121, projection='3d')
    
    # カメラパラメータ
    baseline = 0.1  # 10cm
    f = 500  # ピクセル
    
    # カメラ中心
    C_L = np.array([-baseline/2, 0, 0])
    C_R = np.array([baseline/2, 0, 0])
    
    # 3D点（異なる深度）
    points_3d = np.array([
        [0, 0, 1.0],    # 近い点
        [0.05, 0, 2.0], # 中距離
        [-0.05, 0, 4.0] # 遠い点
    ])
    colors = ['red', 'green', 'blue']
    
    # カメラをプロット
    ax1.scatter(*C_L, color='purple', s=200, marker='o', label='Left Camera')
    ax1.scatter(*C_R, color='orange', s=200, marker='o', label='Right Camera')
    
    # ベースライン
    ax1.plot([C_L[0], C_R[0]], [C_L[1], C_R[1]], [C_L[2], C_R[2]], 
             'k-', linewidth=2, label=f'Baseline = {baseline*100:.0f}cm')
    
    # 3D点と光線
    for i, (pt, color) in enumerate(zip(points_3d, colors)):
        ax1.scatter(*pt, color=color, s=150, marker='*', label=f'Point at Z={pt[2]:.1f}m')
        ax1.plot([C_L[0], pt[0]], [C_L[1], pt[1]], [C_L[2], pt[2]], 
                 color=color, linestyle='--', alpha=0.5)
        ax1.plot([C_R[0], pt[0]], [C_R[1], pt[1]], [C_R[2], pt[2]], 
                 color=color, linestyle='--', alpha=0.5)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z (Depth)')
    ax1.set_title('Stereo Geometry (Top-Down View)', fontsize=12)
    ax1.legend(loc='upper left', fontsize=8)
    ax1.view_init(elev=30, azim=-60)
    
    # 視差と深度の関係
    ax2 = fig.add_subplot(122)
    
    depths = np.linspace(0.5, 10, 100)
    disparities = f * baseline / depths
    
    ax2.plot(depths, disparities, 'b-', linewidth=2)
    ax2.set_xlabel('Depth Z (m)', fontsize=12)
    ax2.set_ylabel('Disparity d (pixels)', fontsize=12)
    ax2.set_title('Disparity vs Depth Relationship\n$d = fb/Z$', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # 3D点の視差をマーク
    for pt, color in zip(points_3d, colors):
        d = f * baseline / pt[2]
        ax2.scatter(pt[2], d, color=color, s=100, zorder=5)
        ax2.annotate(f'Z={pt[2]:.1f}m\nd={d:.1f}px', 
                     xy=(pt[2], d), xytext=(pt[2]+0.5, d+5),
                     fontsize=9, color=color)
    
    ax2.set_xlim(0, 10)
    ax2.set_ylim(0, max(disparities) * 1.1)
    
    plt.tight_layout()
    plt.show()
    
    print(f"パラメータ: f = {f} pixels, baseline = {baseline*100:.0f} cm")
    print("\n視差と深度:")
    for pt in points_3d:
        d = f * baseline / pt[2]
        print(f"  Z = {pt[2]:.1f}m → d = {d:.1f} pixels")

visualize_stereo_geometry()

---

## 3. ステレオ平行化（Rectification）

### 3.1 なぜ平行化が必要か？

一般的なステレオ配置では、カメラは完全に平行ではありません。この場合：

- エピポーラ線が水平ではない
- 対応点探索が2次元探索になる（非効率）

**平行化**により：
- エピポーラ線を**水平**かつ**同じ行**に揃える
- 対応点探索が**1次元の水平探索**に簡略化される

### 3.2 平行化の幾何学

平行化は、両画像にホモグラフィ変換 $\mathbf{H}_L$ と $\mathbf{H}_R$ を適用して、仮想的な平行ステレオ配置を作り出します。

$$\mathbf{x}'_L = \mathbf{H}_L \mathbf{x}_L$$
$$\mathbf{x}'_R = \mathbf{H}_R \mathbf{x}_R$$

変換後：
- 対応点は同じ行（同じy座標）に存在
- $y'_L = y'_R$

In [ ]:
def compute_rectification_homographies(K: np.ndarray, R: np.ndarray, t: np.ndarray
                                       ) -> Tuple[np.ndarray, np.ndarray]:
    """ステレオ平行化のホモグラフィを計算
    
    Bouguet's algorithmの簡略版
    
    Args:
        K: カメラ内部パラメータ（両カメラ共通と仮定）
        R: 右カメラの左カメラに対する回転
        t: 右カメラの左カメラに対する並進
    
    Returns:
        H_L, H_R: 左右画像のホモグラフィ
    """
    # 新しいX軸: ベースライン方向（左から右へ）
    e1 = t / np.linalg.norm(t)
    
    # 新しいY軸: 元のZ軸との外積
    e2 = np.array([-t[1], t[0], 0])
    e2 = e2 / np.linalg.norm(e2) if np.linalg.norm(e2) > 1e-6 else np.array([0, 1, 0])
    
    # 新しいZ軸: X × Y
    e3 = np.cross(e1, e2)
    
    # 新しい回転行列（世界座標から平行化座標へ）
    R_rect = np.vstack([e1, e2, e3])
    
    # 左カメラの回転（補正なし→平行化）
    R_L = R_rect
    
    # 右カメラの回転（R→平行化）
    R_R = R_rect @ R.T
    
    # ホモグラフィ
    H_L = K @ R_L @ np.linalg.inv(K)
    H_R = K @ R_R @ np.linalg.inv(K)
    
    return H_L, H_R

def visualize_rectification_effect():
    """平行化の効果を可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # カメラパラメータ
    K = np.array([
        [500, 0, 320],
        [0, 500, 240],
        [0, 0, 1]
    ])
    
    # 非平行なステレオ配置
    theta = np.radians(10)  # 10度回転
    R = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    t = np.array([0.1, 0.02, 0.01])  # 非理想的な並進
    
    # 基礎行列
    def skew(v):
        return np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    
    E = skew(t) @ R
    F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)
    
    # テスト点
    test_points_L = np.array([
        [100, 200], [200, 150], [300, 250], [400, 300], [500, 200]
    ])
    
    img_size = (640, 480)
    
    # 平行化前：エピポーラ線を描画
    ax = axes[0, 0]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(test_points_L)))
    
    for pt, color in zip(test_points_L, colors):
        ax.scatter(*pt, color=color, s=100, zorder=5)
    
    ax.set_title('Left Image (Before Rectification)', fontsize=12)
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    
    # 右画像：エピポーラ線
    ax = axes[0, 1]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    for pt, color in zip(test_points_L, colors):
        x_h = np.array([pt[0], pt[1], 1])
        l = F @ x_h
        a, b, c = l
        
        if abs(b) > 1e-6:
            x_vals = np.array([0, img_size[0]])
            y_vals = -(a * x_vals + c) / b
            ax.plot(x_vals, y_vals, color=color, linewidth=2)
    
    ax.set_title('Right Image: Epipolar Lines (Non-horizontal)', fontsize=12)
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    
    # 平行化後
    H_L, H_R = compute_rectification_homographies(K, R, t)
    
    # 左画像（平行化後）
    ax = axes[1, 0]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                fill=True, facecolor='#e8f4e8', edgecolor='black'))
    
    transformed_points = []
    for pt, color in zip(test_points_L, colors):
        x_h = np.array([pt[0], pt[1], 1])
        x_rect = H_L @ x_h
        x_rect = x_rect[:2] / x_rect[2]
        transformed_points.append(x_rect)
        
        if 0 <= x_rect[0] <= img_size[0] and 0 <= x_rect[1] <= img_size[1]:
            ax.scatter(*x_rect, color=color, s=100, zorder=5)
            # 水平線を描画
            ax.axhline(y=x_rect[1], color=color, alpha=0.3, linestyle='--')
    
    ax.set_title('Left Image (After Rectification)', fontsize=12)
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    
    # 右画像（平行化後）：水平エピポーラ線
    ax = axes[1, 1]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                fill=True, facecolor='#e8f4e8', edgecolor='black'))
    
    for pt_rect, color in zip(transformed_points, colors):
        if 0 <= pt_rect[1] <= img_size[1]:
            ax.axhline(y=pt_rect[1], color=color, linewidth=2, alpha=0.7)
    
    ax.set_title('Right Image: Epipolar Lines (Horizontal)', fontsize=12)
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    
    plt.suptitle('Effect of Stereo Rectification', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("平行化後、全てのエピポーラ線が水平になります。")
    print("これにより、対応点探索が1次元の水平探索に簡略化されます。")

visualize_rectification_effect()

---

## 4. ステレオマッチング

### 4.1 問題設定

平行化されたステレオ画像ペアに対して、左画像の各ピクセル $(x_L, y)$ に対応する右画像のピクセル $(x_R, y)$ を見つけます。

視差: $d = x_L - x_R$

### 4.2 マッチングコスト

対応点を見つけるため、ピクセル間の類似度（またはコスト）を計算します：

| コスト関数 | 式 | 特徴 |
|-----------|-----|------|
| **SAD** (Sum of Absolute Differences) | $\sum |I_L - I_R|$ | シンプル、高速 |
| **SSD** (Sum of Squared Differences) | $\sum (I_L - I_R)^2$ | 外れ値に敏感 |
| **NCC** (Normalized Cross-Correlation) | $\frac{\sum (I_L - \bar{I}_L)(I_R - \bar{I}_R)}{\sigma_L \sigma_R}$ | 照明変化に強い |
| **Census** | ハミング距離 | エッジに強い |

In [ ]:
def compute_sad(patch1: np.ndarray, patch2: np.ndarray) -> float:
    """SAD (Sum of Absolute Differences)"""
    return np.sum(np.abs(patch1.astype(float) - patch2.astype(float)))

def compute_ssd(patch1: np.ndarray, patch2: np.ndarray) -> float:
    """SSD (Sum of Squared Differences)"""
    diff = patch1.astype(float) - patch2.astype(float)
    return np.sum(diff ** 2)

def compute_ncc(patch1: np.ndarray, patch2: np.ndarray) -> float:
    """NCC (Normalized Cross-Correlation)
    
    Returns: 1 - NCC (コストとして使うため、小さいほど良い)
    """
    p1 = patch1.astype(float).flatten()
    p2 = patch2.astype(float).flatten()
    
    p1_mean = p1 - np.mean(p1)
    p2_mean = p2 - np.mean(p2)
    
    numerator = np.sum(p1_mean * p2_mean)
    denominator = np.sqrt(np.sum(p1_mean**2) * np.sum(p2_mean**2))
    
    if denominator < 1e-6:
        return 1.0
    
    ncc = numerator / denominator
    return 1 - ncc  # コストに変換

def census_transform(image: np.ndarray, window_size: int = 7) -> np.ndarray:
    """Census Transform
    
    各ピクセルを周囲との比較結果のビット列に変換
    """
    h, w = image.shape
    half = window_size // 2
    census = np.zeros((h, w), dtype=np.uint64)
    
    for y in range(half, h - half):
        for x in range(half, w - half):
            center = image[y, x]
            bit_string = 0
            
            for dy in range(-half, half + 1):
                for dx in range(-half, half + 1):
                    if dy == 0 and dx == 0:
                        continue
                    bit_string <<= 1
                    if image[y + dy, x + dx] < center:
                        bit_string |= 1
            
            census[y, x] = bit_string
    
    return census

def hamming_distance(a: int, b: int) -> int:
    """ハミング距離（異なるビットの数）"""
    xor = a ^ b
    count = 0
    while xor:
        count += xor & 1
        xor >>= 1
    return count

print("マッチングコスト関数の実装完了")

In [ ]:
def visualize_matching_costs():
    """マッチングコストの可視化"""
    # サンプルパッチの作成
    np.random.seed(42)
    
    # 基準パッチ（グラデーション + ノイズ）
    x, y = np.meshgrid(np.linspace(0, 1, 9), np.linspace(0, 1, 9))
    base_patch = (x * 200 + np.random.randn(9, 9) * 10).astype(np.uint8)
    
    # 類似パッチ（少しずれた）
    similar_patch = np.roll(base_patch, 1, axis=1)
    
    # 異なるパッチ
    different_patch = (255 - base_patch).astype(np.uint8)
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    
    patches = [base_patch, similar_patch, different_patch]
    titles = ['Reference Patch', 'Similar Patch', 'Different Patch']
    
    for ax, patch, title in zip(axes[0], patches, titles):
        ax.imshow(patch, cmap='gray', vmin=0, vmax=255)
        ax.set_title(title)
        ax.axis('off')
    
    # コスト比較
    costs = {
        'SAD': [],
        'SSD': [],
        'NCC': []
    }
    
    for patch in [similar_patch, different_patch]:
        costs['SAD'].append(compute_sad(base_patch, patch))
        costs['SSD'].append(compute_ssd(base_patch, patch))
        costs['NCC'].append(compute_ncc(base_patch, patch))
    
    # 棒グラフ
    x_pos = np.arange(2)
    width = 0.25
    
    ax = axes[1, 0]
    ax.bar(x_pos - width, costs['SAD'], width, label='SAD', alpha=0.8)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Similar', 'Different'])
    ax.set_ylabel('Cost')
    ax.set_title('SAD Cost')
    ax.legend()
    
    ax = axes[1, 1]
    ax.bar(x_pos, costs['SSD'], width, label='SSD', alpha=0.8, color='orange')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Similar', 'Different'])
    ax.set_title('SSD Cost')
    ax.legend()
    
    ax = axes[1, 2]
    ax.bar(x_pos + width, costs['NCC'], width, label='NCC', alpha=0.8, color='green')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Similar', 'Different'])
    ax.set_title('NCC Cost (1 - correlation)')
    ax.legend()
    
    plt.suptitle('Comparison of Matching Cost Functions', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("\nコスト値:")
    print(f"Similar patch - SAD: {costs['SAD'][0]:.1f}, SSD: {costs['SSD'][0]:.1f}, NCC: {costs['NCC'][0]:.3f}")
    print(f"Different patch - SAD: {costs['SAD'][1]:.1f}, SSD: {costs['SSD'][1]:.1f}, NCC: {costs['NCC'][1]:.3f}")

visualize_matching_costs()

---

## 5. ブロックマッチング

### 5.1 アルゴリズム

```
for each pixel (x, y) in left image:
    extract patch around (x, y)
    
    for each disparity d in [0, max_disparity]:
        extract patch around (x - d, y) in right image
        compute matching cost
    
    disparity[x, y] = argmin(cost)
```

### 5.2 実装

In [ ]:
def block_matching(left: np.ndarray, right: np.ndarray,
                   block_size: int = 9,
                   max_disparity: int = 64,
                   cost_function: str = 'sad') -> np.ndarray:
    """ブロックマッチングによる視差計算
    
    Args:
        left: 左画像 (grayscale)
        right: 右画像 (grayscale)
        block_size: ブロックサイズ（奇数）
        max_disparity: 最大視差
        cost_function: 'sad', 'ssd', または 'ncc'
    
    Returns:
        disparity: 視差マップ
    """
    assert left.shape == right.shape, "画像サイズが一致しません"
    assert block_size % 2 == 1, "ブロックサイズは奇数である必要があります"
    
    h, w = left.shape
    half = block_size // 2
    
    # コスト関数の選択
    if cost_function == 'sad':
        cost_func = compute_sad
    elif cost_function == 'ssd':
        cost_func = compute_ssd
    elif cost_function == 'ncc':
        cost_func = compute_ncc
    else:
        raise ValueError(f"Unknown cost function: {cost_function}")
    
    disparity = np.zeros((h, w), dtype=np.float32)
    
    for y in range(half, h - half):
        for x in range(half + max_disparity, w - half):
            # 左画像のパッチ
            patch_L = left[y-half:y+half+1, x-half:x+half+1]
            
            min_cost = float('inf')
            best_d = 0
            
            for d in range(max_disparity + 1):
                # 右画像のパッチ（左へシフト）
                x_R = x - d
                if x_R - half < 0:
                    continue
                
                patch_R = right[y-half:y+half+1, x_R-half:x_R+half+1]
                
                cost = cost_func(patch_L, patch_R)
                
                if cost < min_cost:
                    min_cost = cost
                    best_d = d
            
            disparity[y, x] = best_d
    
    return disparity

print("ブロックマッチングの実装完了")

In [ ]:
def create_synthetic_stereo_pair(width: int = 320, height: int = 240,
                                  max_disparity: int = 32) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """合成ステレオ画像ペアを生成
    
    Returns:
        left: 左画像
        right: 右画像
        gt_disparity: 真の視差マップ
    """
    np.random.seed(42)
    
    # 背景（テクスチャ付き）
    background = np.random.randint(100, 200, (height, width), dtype=np.uint8)
    background = ndimage.gaussian_filter(background.astype(float), sigma=2).astype(np.uint8)
    
    # 深度マップ（複数の長方形オブジェクト）
    gt_disparity = np.ones((height, width), dtype=np.float32) * 5  # 背景視差
    
    # オブジェクトを配置
    objects = [
        {'x': 50, 'y': 50, 'w': 80, 'h': 100, 'disp': 25},   # 近いオブジェクト
        {'x': 180, 'y': 80, 'w': 60, 'h': 80, 'disp': 15},   # 中距離
        {'x': 100, 'y': 150, 'w': 100, 'h': 50, 'disp': 20}, # 手前の棚
    ]
    
    left = background.copy()
    
    for obj in objects:
        x, y, w, h, d = obj['x'], obj['y'], obj['w'], obj['h'], obj['disp']
        
        # オブジェクトのテクスチャ
        texture = np.random.randint(50, 150, (h, w), dtype=np.uint8)
        # チェッカーパターンを追加
        checker = np.indices((h, w)).sum(axis=0) % 20 < 10
        texture[checker] += 30
        
        left[y:y+h, x:x+w] = texture
        gt_disparity[y:y+h, x:x+w] = d
    
    # 右画像を視差に基づいてシフト
    right = np.zeros_like(left)
    
    for y in range(height):
        for x in range(width):
            d = int(gt_disparity[y, x])
            x_R = x - d
            if 0 <= x_R < width:
                right[y, x_R] = left[y, x]
    
    # 穴埋め（簡易的）
    right = ndimage.grey_dilation(right, size=3)
    right = ndimage.gaussian_filter(right.astype(float), sigma=0.5).astype(np.uint8)
    
    return left, right, gt_disparity

# 合成データの生成
left_img, right_img, gt_disp = create_synthetic_stereo_pair()

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].imshow(left_img, cmap='gray')
axes[0].set_title('Left Image')
axes[0].axis('off')

axes[1].imshow(right_img, cmap='gray')
axes[1].set_title('Right Image')
axes[1].axis('off')

im = axes[2].imshow(gt_disp, cmap='jet')
axes[2].set_title('Ground Truth Disparity')
axes[2].axis('off')
plt.colorbar(im, ax=axes[2], label='Disparity (pixels)')

plt.suptitle('Synthetic Stereo Pair', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ブロックマッチングの実行
print("ブロックマッチングを実行中...")

disparity_sad = block_matching(left_img, right_img, 
                                block_size=9, max_disparity=32, 
                                cost_function='sad')

print("完了!")

# 結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].imshow(gt_disp, cmap='jet', vmin=0, vmax=32)
axes[0].set_title('Ground Truth Disparity')
axes[0].axis('off')

axes[1].imshow(disparity_sad, cmap='jet', vmin=0, vmax=32)
axes[1].set_title('Estimated Disparity (Block Matching)')
axes[1].axis('off')

# 誤差マップ
error = np.abs(disparity_sad - gt_disp)
im = axes[2].imshow(error, cmap='hot', vmin=0, vmax=10)
axes[2].set_title('Absolute Error')
axes[2].axis('off')
plt.colorbar(im, ax=axes[2], label='Error (pixels)')

plt.suptitle('Block Matching Results', fontsize=14)
plt.tight_layout()
plt.show()

# 精度評価
valid_mask = gt_disp > 0
mae = np.mean(error[valid_mask])
bad_pixels = np.sum(error[valid_mask] > 3) / np.sum(valid_mask) * 100

print(f"\n精度評価:")
print(f"  MAE (Mean Absolute Error): {mae:.2f} pixels")
print(f"  Bad pixels (error > 3px): {bad_pixels:.1f}%")

---

## 6. Semi-Global Matching (SGM)

### 6.1 ブロックマッチングの問題点

- テクスチャレス領域で不安定
- オクルージョン境界でノイジー
- 局所的な最適化のみ（グローバルな整合性なし）

### 6.2 SGMの考え方

**Semi-Global Matching (SGM)** は、複数の方向からのコスト集約により、グローバルな整合性を考慮します。

#### コスト集約

方向 $\mathbf{r}$ に沿った累積コスト：

$$L_\mathbf{r}(\mathbf{p}, d) = C(\mathbf{p}, d) + \min \begin{cases}
L_\mathbf{r}(\mathbf{p}-\mathbf{r}, d) \\
L_\mathbf{r}(\mathbf{p}-\mathbf{r}, d-1) + P_1 \\
L_\mathbf{r}(\mathbf{p}-\mathbf{r}, d+1) + P_1 \\
\min_i L_\mathbf{r}(\mathbf{p}-\mathbf{r}, i) + P_2
\end{cases}$$

ここで：
- $P_1$: 小さな視差変化へのペナルティ
- $P_2$: 大きな視差変化へのペナルティ（$P_2 > P_1$）

#### 全方向からの集約

$$S(\mathbf{p}, d) = \sum_\mathbf{r} L_\mathbf{r}(\mathbf{p}, d)$$

最終視差：

$$d(\mathbf{p}) = \arg\min_d S(\mathbf{p}, d)$$

In [ ]:
def compute_cost_volume(left: np.ndarray, right: np.ndarray,
                        max_disparity: int, block_size: int = 5) -> np.ndarray:
    """コストボリュームを計算（SAD）
    
    Returns:
        cost_volume: shape (H, W, max_disparity+1)
    """
    h, w = left.shape
    half = block_size // 2
    cost_volume = np.full((h, w, max_disparity + 1), np.inf, dtype=np.float32)
    
    for d in range(max_disparity + 1):
        for y in range(half, h - half):
            for x in range(half + d, w - half):
                patch_L = left[y-half:y+half+1, x-half:x+half+1]
                patch_R = right[y-half:y+half+1, x-d-half:x-d+half+1]
                cost_volume[y, x, d] = compute_sad(patch_L, patch_R)
    
    return cost_volume

def sgm_aggregate_direction(cost_volume: np.ndarray, 
                             direction: Tuple[int, int],
                             P1: float = 10, P2: float = 150) -> np.ndarray:
    """1方向のSGMコスト集約
    
    Args:
        cost_volume: (H, W, D)
        direction: (dy, dx)
        P1: 小さな視差変化のペナルティ
        P2: 大きな視差変化のペナルティ
    """
    h, w, d_max = cost_volume.shape
    dy, dx = direction
    
    L = np.zeros_like(cost_volume)
    
    # 開始位置の決定
    if dy > 0:
        y_range = range(h)
    else:
        y_range = range(h - 1, -1, -1)
    
    if dx > 0:
        x_range = range(w)
    else:
        x_range = range(w - 1, -1, -1)
    
    for y in y_range:
        for x in x_range:
            py, px = y - dy, x - dx
            
            if 0 <= py < h and 0 <= px < w:
                L_prev = L[py, px]
                
                for d in range(d_max):
                    # 4つの候補からの最小コスト
                    costs = [L_prev[d]]  # 同じ視差
                    
                    if d > 0:
                        costs.append(L_prev[d-1] + P1)  # 視差-1
                    
                    if d < d_max - 1:
                        costs.append(L_prev[d+1] + P1)  # 視差+1
                    
                    costs.append(np.min(L_prev) + P2)  # 大きなジャンプ
                    
                    L[y, x, d] = cost_volume[y, x, d] + min(costs) - np.min(L_prev)
            else:
                L[y, x] = cost_volume[y, x]
    
    return L

def semi_global_matching(left: np.ndarray, right: np.ndarray,
                          max_disparity: int = 32,
                          block_size: int = 5,
                          P1: float = 10, P2: float = 150) -> np.ndarray:
    """Semi-Global Matching (SGM)
    
    8方向からのコスト集約
    """
    # コストボリュームの計算
    print("  コストボリューム計算中...")
    cost_volume = compute_cost_volume(left, right, max_disparity, block_size)
    
    # 8方向
    directions = [
        (0, 1),   # 右
        (0, -1),  # 左
        (1, 0),   # 下
        (-1, 0),  # 上
        (1, 1),   # 右下
        (1, -1),  # 左下
        (-1, 1),  # 右上
        (-1, -1)  # 左上
    ]
    
    # 各方向からの集約
    print("  8方向からのコスト集約中...")
    S = np.zeros_like(cost_volume)
    
    for i, direction in enumerate(directions):
        L = sgm_aggregate_direction(cost_volume, direction, P1, P2)
        S += L
    
    # 視差の決定（WTA: Winner-Take-All）
    disparity = np.argmin(S, axis=2).astype(np.float32)
    
    return disparity

print("SGMの実装完了")

In [ ]:
# SGMの実行
print("Semi-Global Matching を実行中...")

disparity_sgm = semi_global_matching(left_img, right_img, 
                                      max_disparity=32, block_size=5,
                                      P1=10, P2=150)

print("完了!")

# 結果の比較
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(gt_disp, cmap='jet', vmin=0, vmax=32)
axes[0, 0].set_title('Ground Truth', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(disparity_sad, cmap='jet', vmin=0, vmax=32)
axes[0, 1].set_title('Block Matching (SAD)', fontsize=12)
axes[0, 1].axis('off')

axes[1, 0].imshow(disparity_sgm, cmap='jet', vmin=0, vmax=32)
axes[1, 0].set_title('Semi-Global Matching', fontsize=12)
axes[1, 0].axis('off')

# 誤差比較
error_sad = np.abs(disparity_sad - gt_disp)
error_sgm = np.abs(disparity_sgm - gt_disp)

valid_mask = gt_disp > 0
mae_sad = np.mean(error_sad[valid_mask])
mae_sgm = np.mean(error_sgm[valid_mask])

methods = ['Block Matching', 'SGM']
maes = [mae_sad, mae_sgm]
colors = ['blue', 'green']

axes[1, 1].bar(methods, maes, color=colors, alpha=0.7)
axes[1, 1].set_ylabel('MAE (pixels)', fontsize=12)
axes[1, 1].set_title('Error Comparison', fontsize=12)
for i, (method, mae) in enumerate(zip(methods, maes)):
    axes[1, 1].text(i, mae + 0.1, f'{mae:.2f}', ha='center', fontsize=11)

plt.suptitle('Comparison: Block Matching vs SGM', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n精度比較:")
print(f"  Block Matching MAE: {mae_sad:.2f} pixels")
print(f"  SGM MAE: {mae_sgm:.2f} pixels")

---

## 7. 視差から深度への変換

### 7.1 深度マップの生成

$$Z = \frac{f \cdot b}{d}$$

### 7.2 3D点群の生成

各ピクセル $(u, v)$ に対して、深度 $Z$ から3D座標を復元：

$$X = \frac{(u - c_x) \cdot Z}{f}$$
$$Y = \frac{(v - c_y) \cdot Z}{f}$$

In [ ]:
def disparity_to_depth(disparity: np.ndarray, 
                       focal_length: float, 
                       baseline: float) -> np.ndarray:
    """視差マップから深度マップへの変換
    
    Args:
        disparity: 視差マップ（ピクセル）
        focal_length: 焦点距離（ピクセル）
        baseline: ベースライン（メートル）
    
    Returns:
        depth: 深度マップ（メートル）
    """
    # ゼロ除算を避ける
    disparity_safe = np.maximum(disparity, 0.1)
    depth = (focal_length * baseline) / disparity_safe
    
    # 無効な視差（0以下）をマスク
    depth[disparity <= 0] = 0
    
    return depth

def depth_to_pointcloud(depth: np.ndarray, K: np.ndarray,
                        image: Optional[np.ndarray] = None) -> Tuple[np.ndarray, Optional[np.ndarray]]:
    """深度マップから3D点群を生成
    
    Args:
        depth: 深度マップ（メートル）
        K: カメラ内部パラメータ
        image: 色情報（オプション）
    
    Returns:
        points: 3D点群 (N, 3)
        colors: 色情報 (N, 3) or None
    """
    h, w = depth.shape
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    # ピクセル座標のグリッド
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    
    # 有効な深度のマスク
    valid = depth > 0
    
    # 3D座標の計算
    Z = depth[valid]
    X = (u[valid] - cx) * Z / fx
    Y = (v[valid] - cy) * Z / fy
    
    points = np.vstack([X, Y, Z]).T
    
    # 色情報
    if image is not None:
        if len(image.shape) == 2:
            colors = np.stack([image[valid]] * 3, axis=1) / 255.0
        else:
            colors = image[valid] / 255.0
    else:
        colors = None
    
    return points, colors

# カメラパラメータ（仮定）
focal_length = 500  # pixels
baseline = 0.1      # 10cm

K = np.array([
    [focal_length, 0, left_img.shape[1] / 2],
    [0, focal_length, left_img.shape[0] / 2],
    [0, 0, 1]
])

# 深度マップの計算
depth_map = disparity_to_depth(disparity_sgm, focal_length, baseline)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].imshow(disparity_sgm, cmap='jet')
axes[0].set_title('Disparity Map')
axes[0].axis('off')
plt.colorbar(axes[0].images[0], ax=axes[0], label='Disparity (px)')

# 深度の可視化（適切な範囲にクリップ）
depth_display = np.clip(depth_map, 0, 5)  # 0-5m
im = axes[1].imshow(depth_display, cmap='viridis_r')
axes[1].set_title('Depth Map')
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], label='Depth (m)')

# 深度のヒストグラム
valid_depths = depth_map[(depth_map > 0) & (depth_map < 5)]
axes[2].hist(valid_depths, bins=50, edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Depth (m)')
axes[2].set_ylabel('Count')
axes[2].set_title('Depth Distribution')

plt.suptitle('Disparity to Depth Conversion', fontsize=14)
plt.tight_layout()
plt.show()

print(f"深度範囲: {valid_depths.min():.2f}m - {valid_depths.max():.2f}m")
print(f"平均深度: {valid_depths.mean():.2f}m")

In [ ]:
def visualize_3d_reconstruction(depth: np.ndarray, K: np.ndarray, 
                                 image: np.ndarray, max_points: int = 5000):
    """3D点群の可視化"""
    points, colors = depth_to_pointcloud(depth, K, image)
    
    # ダウンサンプリング
    if len(points) > max_points:
        indices = np.random.choice(len(points), max_points, replace=False)
        points = points[indices]
        if colors is not None:
            colors = colors[indices]
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # 深度で色付け
    scatter = ax.scatter(points[:, 0], points[:, 2], -points[:, 1], 
                         c=points[:, 2], cmap='viridis_r', s=1, alpha=0.5)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Z (Depth, m)')
    ax.set_zlabel('Y (m)')
    ax.set_title('3D Point Cloud from Stereo', fontsize=14)
    
    plt.colorbar(scatter, label='Depth (m)', shrink=0.5)
    
    # 視点調整
    ax.view_init(elev=20, azim=-60)
    
    plt.tight_layout()
    plt.show()
    
    print(f"点群サイズ: {len(points)} 点")

visualize_3d_reconstruction(depth_map, K, left_img)

---

## 8. 実践的な考慮事項

### 8.1 ステレオマッチングの課題

| 課題 | 説明 | 対処法 |
|------|------|--------|
| **オクルージョン** | 片方の画像でのみ見える領域 | Left-Right Consistency Check |
| **テクスチャレス領域** | マッチングが曖昧 | 大きなウィンドウ、SGM |
| **反復パターン** | 複数の候補が存在 | グローバル最適化 |
| **照明変化** | 左右画像の輝度差 | Census Transform, NCC |

### 8.2 Left-Right Consistency Check

1. 左画像から右画像への視差 $d_L$ を計算
2. 右画像から左画像への視差 $d_R$ を計算
3. 一致しない点を除外: $|d_L(x) - d_R(x - d_L(x))| > \tau$

In [ ]:
def left_right_consistency_check(disp_L: np.ndarray, disp_R: np.ndarray,
                                  threshold: float = 1.0) -> np.ndarray:
    """Left-Right Consistency Check
    
    Args:
        disp_L: 左→右の視差マップ
        disp_R: 右→左の視差マップ
        threshold: 許容誤差（ピクセル）
    
    Returns:
        valid_mask: 有効なピクセルのマスク
    """
    h, w = disp_L.shape
    valid_mask = np.zeros((h, w), dtype=bool)
    
    for y in range(h):
        for x in range(w):
            d = int(round(disp_L[y, x]))
            x_R = x - d
            
            if 0 <= x_R < w:
                d_R = disp_R[y, x_R]
                
                if abs(d - d_R) <= threshold:
                    valid_mask[y, x] = True
    
    return valid_mask

# 右→左の視差も計算（簡略版：左右を入れ替え）
print("右→左の視差マップを計算中...")
disparity_RL = block_matching(right_img, left_img, 
                               block_size=9, max_disparity=32, 
                               cost_function='sad')
print("完了!")

# Consistency Check
valid_mask = left_right_consistency_check(disparity_sad, disparity_RL, threshold=1.0)

# 結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].imshow(disparity_sad, cmap='jet', vmin=0, vmax=32)
axes[0].set_title('Original Disparity')
axes[0].axis('off')

axes[1].imshow(valid_mask, cmap='gray')
axes[1].set_title(f'Valid Pixels ({np.sum(valid_mask) / valid_mask.size * 100:.1f}%)')
axes[1].axis('off')

# フィルタリング後
disparity_filtered = disparity_sad.copy()
disparity_filtered[~valid_mask] = 0
axes[2].imshow(disparity_filtered, cmap='jet', vmin=0, vmax=32)
axes[2].set_title('After L-R Check')
axes[2].axis('off')

plt.suptitle('Left-Right Consistency Check', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n有効ピクセル率: {np.sum(valid_mask) / valid_mask.size * 100:.1f}%")

---

## 9. まとめと次のステップ

### 学んだこと

1. **ステレオビジョンの原理**: ベースライン、視差、深度の関係
2. **ステレオ平行化**: エピポーラ線を水平に揃え、1D探索に簡略化
3. **マッチングコスト**: SAD, SSD, NCC, Census
4. **ブロックマッチング**: 局所的な視差計算
5. **SGM**: グローバルな整合性を考慮した視差計算
6. **深度復元**: 視差から深度へ、3D点群の生成

### 重要な数式

| 概念 | 数式 |
|------|------|
| 視差 | $d = x_L - x_R$ |
| 深度 | $Z = \frac{f \cdot b}{d}$ |
| 3D座標 | $X = \frac{(u - c_x) \cdot Z}{f}$ |

### 次のノートブック

**57. 三角測量と3D復元**では：
- DLT（Direct Linear Transform）による三角測量
- 最小二乗法による最適化
- 誤差解析と精度向上

---

## 10. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. 視差と深度の関係式 $Z = fb/d$ を導出してください。

2. ステレオ平行化（Rectification）の目的は何ですか？

3. SAD と NCC のマッチングコストの違いは？どのような状況でNCCが有利ですか？

4. SGMがブロックマッチングより優れている点は何ですか？

5. オクルージョン領域で視差推定が困難な理由は？

6. Left-Right Consistency Check の原理と目的を説明してください。

7. 視差が0の場合、深度はどうなりますか？これは物理的に何を意味しますか？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. Z = fb/d の導出:
       - 三角形の相似より: x_L/f = X/Z, x_R/f = (X-b)/Z
       - 視差: d = x_L - x_R = f(X/Z) - f(X-b)/Z = fb/Z
       - よって: Z = fb/d
    
    2. 平行化の目的:
       - エピポーラ線を水平かつ同じ行に揃える
       - 対応点探索を2D探索から1D水平探索に簡略化
       - 計算効率の大幅な向上
    
    3. SAD vs NCC:
       - SAD: 絶対差の和、高速だが照明変化に敏感
       - NCC: 正規化相互相関、照明変化に強いが計算コスト高い
       - NCCが有利: 左右カメラの露出が異なる場合、影がある場合
    
    4. SGMの利点:
       - 複数方向からのコスト集約でグローバルな整合性を考慮
       - テクスチャレス領域でより安定
       - 視差の滑らかさを保ちつつエッジを保存
    
    5. オクルージョンの問題:
       - オクルージョン領域は一方の画像でのみ見える
       - 対応点が存在しないため、マッチングが不可能
       - 誤った対応を見つけてしまう可能性
    
    6. L-R Consistency Check:
       - 原理: 左→右と右→左の視差が一致するか確認
       - 目的: オクルージョンや誤マッチングの検出・除去
       - 一致しない点は信頼性が低いと判断
    
    7. 視差が0の場合:
       - Z = fb/0 = ∞（無限遠）
       - 物理的意味: 非常に遠い物体（または平行移動がない場合の全点）
       - 実際には数値的に扱えないため、最小視差を設定することが多い
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [55. エピポーラ幾何の理論](55_epipolar_geometry_theory_v1.ipynb)
- **次のノートブック**: [57. 三角測量と3D復元](57_triangulation_3d_reconstruction_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)